In [ ]:
### ------------------- Notebook Parameters ------------------- ###

PARAMS = {'dow':'friday',
          'country':'All',
          'type':'All'
         }

assert PARAMS['dow'] in ['friday', 'saturday', 'sunday', 'All']
assert PARAMS['country'] in ['DE', 'US', 'FR', 'All']
assert PARAMS['type'] in ['Type A', 'Type B', 'All']


### ----------------------------------------------------------- ###

# Analysis

Template notebook for analysis, including some commonly used starter code and formatting techniques.
- generated from https://github.com/pawlodkowski/analysis_template

## Contents<a class="anchor" id="contents"></a>

1. [Background](#background)

2. [Executive Summary](#summary)

3. [Q1](#q1)

    a. [Q1a](#1a)
    
    b. [Q1b](#1b)

4. [Q2](#q2)

    a. [Q2a](#2a)
    
    b. [Q2b](#2b)
    
5. [Appendix](#Appendix)

6. [Footnotes](#Footnotes)

In [ ]:
from datetime import datetime
import os
import re
from typing import List, Tuple

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import sqlalchemy
import yaml
from IPython.display import Markdown, display, display_html
from scipy import stats

pd.options.plotting.backend = "plotly"

def get_db_url(file='database.yml', conn='dwh') -> str:
    """
    Returns database URL from credential variables found in .yml file.
    """
    path_options = [os.path.expanduser('~') + '/.rport/' + file,
                    os.getcwd() + '/' + file,
                    '/root/' + file]
    for opt in path_options:
        if os.path.isfile(opt):
            with open(opt) as data:
                creds = yaml.safe_load(data)
            break
    else:
        err = '\nCredentials file not found from the list of options:\n'
        for opt in path_options:
            err += f'\t- {opt}\n'
        raise Exception(err)
         
    c = creds.get(conn)
    if not c:
        raise Exception(f'\nNo data found for connnection named {conn}!\n')
        
    db_url = f"""\
            {c['driver'].lower()}://{c['username']}:\
            {c['password']}@{c['host']}:{c['port']}/{c['database']}"""
    return re.sub('\s+', '', db_url)

def filter_by_params(df:pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for param, val in PARAMS.items():
        if val != 'All':
            df = df.query(f'{param} == "{val}"')
    return df

In [ ]:
DB_URL = get_db_url()
CON = sqlalchemy.create_engine(DB_URL)

#modify or delete
QUERY_PARAMS = {'start_date':'2022-01-01 00:00:00'}
              
SQL = """
with dummy_query as (
  select 
    date_trunc('day', dates)::date as day,
    btrim(to_char(dates, 'day')) as dow,
    (array['DE', 'US', 'FR'])[floor(random() * 3 + 1)] as country,
    (array['Type A', 'Type B'])[floor(random() * 2 + 1)] as type,
    round((random() * 50 + 1)::numeric, 0) as num,
    round((random() * 1)::numeric, 2) as frac
  from generate_series(%(start_date)s::timestamp,
                       %(start_date)s::timestamp + interval '60 days', 
                       '1 day'::interval
                       ) dates
)
select * from dummy_query
"""
df = pd.read_sql(SQL, CON, params=QUERY_PARAMS)

In [ ]:
df = filter_by_params(df)

In [ ]:
df.set_index('day').head()\
  .style.background_gradient(cmap='Greens',
                             subset='frac',
                             vmax=1.0
                            )\
  .bar(align='mid',
       subset='num',
       color=['#ff9994', '#c7eaf2']
      )\
  .format({'num':'{:,.0f}',
           'frac':'{:.1%}'
          })\
  .set_properties(**{'width': '100px'})\
  .set_table_styles([{'selector':'tr .index_name',
                      'props': 'font-style: italic; color: dimgrey; font-weight:bold;'
                     }],
                    axis=None,
                    overwrite=True
                   )\
  .set_caption(f'<b>Something Interesting</b><br><i>(like, very interesting)</i>')\
  .set_table_styles([{'selector': 'caption',
                      'props': 'caption-side: top; font-size:1.25em; margin-bottom:10px'}],
                    overwrite=False
                   )                       

## Background<a class="anchor" id="background"></a>:

In [ ]:
text = f"""
Here is the text for the background. 

When referencing more specific things (like technical definitions) that don't belong in a high-level summary,
you can reference footnotes<a href="#footnote1"><sup>1</sup></a>.
"""
display(Markdown(text))

## Executive Summary<a class="anchor" id="summary"></a>:

In [ ]:
#placeholder (content will be retroactively moved here during generation of HTML report)
display_html('<div id="executive-summary-placeholder"></div>', raw=True)

## Q1 (Heading Level 2) <a class="anchor" id="q1"></a>

In [ ]:
text = f"""
Here is the text for the first question. And here is the second footnote<a href="#footnote2"><sup>2</sup></a>.

The question-level (i.e. a section) should be heading level 2 (i.e. `<h2>` in HTML or `##` in Markdown), **along with the the executive summary and
table of contents**.
- This is essentially the "top-level", as `<h1>` is only reserved for the title of the report.
"""
display(Markdown(text))

### Q1a (Heading Level 3) <a class="anchor" id="1a"></a>

In [ ]:
text = f"""
Sub-sections within a question should be heading level 3 (i.e. `<h3>` in HTML or `###` in Markdown).
"""
display(Markdown(text))

#### Sub-Sub-Section (Heading Level 4)<a class="anchor" id="descriptive_name"></a>

In [ ]:
text = f"""
Sub-sections within a sub-section within a question should be heading level 4 (i.e. `<h4>` in HTML or `####` in Markdown).
"""
display(Markdown(text))

#### Sub-Sub-Sub-Section (Heading Level 5)<a class="anchor" id="descriptive_name"></a>

In [ ]:
text = f"""
The deepest level allowed is heading level 5 (i.e. `<h5>` in HTML or `#####` in Markdown). 

- This is the lowest heading level that is targeted by my 
[flowkey nbconvert template](https://github.com/pawlodkowski/nbconvert_flowkey/blob/master/share/jupyter/nbconvert/templates/flowkey/index.html.j2) and
given styling and anchor links for easier navigating / internal hyperlinking.
- Using anything lower will just be turned into regular text in the HTML report (_which won't look nice_).
"""
display(Markdown(text))

### Q1b (Heading Level 3) <a class="anchor" id="1b"></a> 

In [ ]:
text = f"""
some text here...
"""
display(Markdown(text))

## Q2 (Heading Level 2)<a class="anchor" id="q2"></a>

In [ ]:
text = f"""
some text here...
"""
display(Markdown(text))

### Q2a (Heading Level 3)<a class="anchor" id="2a"></a>

In [ ]:
text = f"""
some text here...
"""
display(Markdown(text))

### Q2b (Heading Level 3) <a class="anchor" id="2b"></a> 

In [ ]:
text = f"""
some text here...
"""
display(Markdown(text))

## Appendix<a class="anchor" id="Appendix"></a>

In [ ]:
text = f"""
This is the text for the appendix. 
"""
display(Markdown(text))

## Footnotes<a class="anchor" id="Footnotes"></a>

**<a name="footnote1">1</a>:** _Here is the text for footnote 1._

**<a name="footnote2">2</a>:** _Here is the text for footnote 2._

---

In [ ]:
exec_summary = f"""
<div id="executive-summary-text">
<p>This report goes into considerable detail, so here are the highlights:</p>

    <ul>
        <li>
            ⭐️ First point: The average frac val is <b>{df.frac.mean():,.1%}</b>. Cool!
        </li>
    </ul>
    <blockquote>
        <p>
            <em>
                see section: 
                <a href="#q1" target="_self" rel="noopener">Q1</a>
            </em>
        </p>
    </blockquote>
    
    <ul>
        <li>
            ⭐️ Second point: The sum of all the nums in the data set is <b>{df.num.sum():,.0f}</b>! Fantastic.
        </li>
    </ul>
    <blockquote>
        <p>
            <em>
                see section: 
                <a href="#q2" target="_self" rel="noopener">Q2</a>
            </em>
        </p>
    </blockquote>
    
<hr>
</div>
"""
display_html(exec_summary, raw=True)

In [ ]:
display_html(
    '<div id="timestamp_container"\
    style="display:flex; justify-content:center; align-items:center; font-size:smaller; color:dimgray; margin:5px;">\
    <p><i>Report generated at: <span id="timestamp" style="font-weight:bold;">{} (UTC)</span></i></p></div>'\
    .format(datetime.utcnow().strftime('%Y-%m-%d @ %H:%M:%S')),
    raw=True
)